## TODO
- Social graph
- Simplex geometry
- Simplex sampler
- Other optimality criteria
- Same RNG for all rand() calls

In [115]:
include("./pso.jl")
using .PSO

include("../../src/optim/optimality_criteria.jl")
using .OptimalityCriteria

include("../../src/doe/designs.jl")
using .Designs

include("../../src/util/tensor_ops.jl")
using .TensorOps

In [193]:
# Parameters
N = 9
K = 3
S = 150
max_iter = 5
relTol = 0
maxStag = 500

500

In [194]:
# Define experimental optimization problem
initializer = Designs.initialize_hypercube(N, K)
constraints = PSO.hypercube_constraints()
objective = OptimalityCriteria.D

# Define PSO and simulation runner parameters
pso_params = PSO.default_hyperparams()
runner_params = PSO.runner_params(max_iter, maxStag, relTol)

Main.PSO.PSOTypes.RunnerParams(5, 500, 0.0)

In [195]:
swarm = PSO.initialize_swarm(initializer, objective, constraints, pso_params)
final_state = PSO.optimize(swarm, runner_params)

Iteration: 1, Best score: 0.013301779696816902
Iteration: 2, Best score: 0.005701163900705826
Iteration: 3, Best score: 0.005701163900705826
Iteration: 4, Best score: 0.005701163900705826
Iteration: 5, Best score: 0.005701163900705826


Main.PSO.PSOTypes.RunnerState(Main.PSO.PSOTypes.Swarm(Main.PSO.PSOTypes.ParticleState([1.0 0.435962441121743 … 0.03195473065852045 -0.8107598762778794; 0.2185600007352564 -0.5669066256317447 … 0.43534819214427856 0.05228874490306035; … ; 0.6769859313067228 1.0 … 0.0615113280104137 0.018822761161261914; 0.8830163896076 0.9598084178090108 … 0.16290179692733942 -1.0;;; 1.0 -0.07138697748061199 … 0.6268087232328707 0.21668232896869377; 0.10434045527192182 0.0763954297160866 … 0.42289423667750226 0.3332138044065053; … ; 1.0 -0.5036659922424886 … 0.3014756125092256 0.41858994061405524; 0.7264419680172665 -0.22725982171938475 … -0.08839997287109858 0.34446139883367055;;; -0.05106558822015361 -0.9493053977171266 … 0.571358186054177 -0.972513927254461; -0.5093740232223626 -0.8457784680250056 … 0.5957065914878596 -0.5105191874867502; … ; -0.5509241446271065 -0.3782492182994027 … 0.9932206622360695 0.11567976332583518; -0.6143430194149987 -0.5947757921818166 … 0.9021087814460367 -0.48156113872046

In [196]:
final_state = PSO.optimize(final_state.swarm, runner_params)
optimal_design, fitness_score = PSO.get_result(final_state)
fitness_score

Iteration: 1, Best score: 0.005701163900705826
Iteration: 2, Best score: 0.005701163900705826
Iteration: 3, Best score: 0.005701163900705826
Iteration: 4, Best score: 0.005701163900705826
Iteration: 5, Best score: 0.005701163900705826


0.005701163900705826

In [197]:
optimal_design

9×3 Matrix{Float64}:
  1.0        1.0       -0.403654
  1.0       -0.36826   -0.729527
 -0.45228   -1.0       -1.0
 -0.951168   1.0        0.857211
  0.21222    0.483791   1.0
  0.190712  -0.980055   0.125976
  0.753826  -1.0        1.0
  0.199602   0.784604   0.850038
 -1.0        0.815061  -1.0

## Social

In [227]:
S = 3

3

In [253]:
scores = vec([.1, .2, .3])

3-element Vector{Float64}:
 0.1
 0.2
 0.3

In [252]:
neighbs = PSO.create_adjacency_matrix(S, 1)

3×3 SparseMatrixCSC{Bool, Int64} with 5 stored entries:
 1  ⋅  1
 ⋅  1  1
 ⋅  ⋅  1

In [254]:
neighbor_scores = neighbs .* transpose(scores)
normalized_neighbor_scores = neighbor_scores ./ sum(neighbor_scores, dims=2)

3×3 SparseMatrixCSC{Float64, Int64} with 9 stored entries:
 0.25  0.0  0.75
 0.0   0.4  0.6
 0.0   0.0  1.0

In [276]:
velocity .* reshape(normalized_neighbor_scores, S, S, 1)

3×3×2 Array{Float64, 3}:
[:, :, 1] =
 0.25  0.0  0.75
 0.0   0.4  0.6
 0.0   0.0  1.0

[:, :, 2] =
 0.25  0.0  0.75
 0.0   0.4  0.6
 0.0   0.0  1.0

In [290]:
size(velocity)

(3, 3, 2)

In [305]:
result = Array{Float64}(undef, S, S, N, K)

# Apply the adjacency matrix to each slice
for (k, r) in enumerate(eachrow(normalized_neighbor_scores))
   tmp = velocity .* r
    result[k, :, :, :] = tmp
end

In [309]:
result[1, 3, :, :]

3×2 Matrix{Float64}:
 0.75  0.75
 0.75  0.75
 0.75  0.75